In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# 读取输入数据集
ds = dataiku.Dataset("CAC_40_Time_Series_CAC_40__FCHI__")
df = ds.get_dataframe()

# 1. 解析日期列
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
df = df.sort_values('Date')

# 2. 清洗数值列（去除千分位逗号，转换为 float）
cols_to_clean = ['Price']
for col in cols_to_clean:
    df[col] = df[col].str.replace(',', '').str.strip()
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. 计算 log-return
df['log_return'] = np.log(df['Price'] / df['Price'].shift(1))

# 4. 写入输出数据集
output = dataiku.Dataset("Test")
output.write_with_schema(df)

# 5. 绘图：log-return 时间序列图
plt.figure(figsize=(12, 5))
plt.plot(df['Date'], df['log_return'], color='brown', label='Monthly log-change in index level')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.7)

plt.title("Monthly Log-Returns of CAC 40 Total Return Index", fontsize=14, weight="bold")
plt.xlabel("Date")
plt.ylabel("Log-return")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.gca().xaxis.set_major_locator(mdates.YearLocator(2))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.tight_layout()

# 6. 保存图像（保存路径视你权限决定）
folder = dataiku.Folder("Managed Folder")  # 替换为你文件夹的ID
folder_path = folder.get_info()["path"]
plt.savefig(folder_path + "/log_returns_cac40.png", dpi=300)